In [1]:
!pip install shap lime ollama python-dotenv
#!pip install "numpy==1.26.4" --force-reinstall
#!pip install "shap<0.50" lime ollama python-dotenv



In [2]:
import os
import sys
import importlib

# Dacă rulezi notebook-ul din folderul demo/, cwd = .../xai-nlg-framework/demo
# Deci root-ul proiectului e un nivel mai sus:
ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
if ROOT_DIR not in sys.path:
    sys.path.append(ROOT_DIR)

import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

import src.pipeline
importlib.reload(src.pipeline)

from config.settings import FrameworkConfig
from src.pipeline import XAINLGPipeline
from src.nlg.few_shot_generator import FewShotGenerator
from src.nlg.ollama_client import ollama_llm_call


dataset + model

In [3]:
# 1. Dataset
data = load_breast_cancer()
X = data.data
y = data.target
feature_names = list(data.feature_names)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 2. Model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 3. Config + pipeline
config = FrameworkConfig(verbose = True)
config.explainer.top_k_features = 5 
pipeline = XAINLGPipeline(
    model,
    X_train,
    feature_names,
    config=config,
)


len(X_train), len(X_test), len(feature_names)


(455, 114, 30)

pipeline + explicație SHAP

In [4]:
# alegem o instanță de explicat
instance = X_test[0]
raw_pred = int(model.predict(instance.reshape(1, -1))[0])
print("Raw prediction:", raw_pred)

# generăm explicația SHAP (fără NLG intern)
result = pipeline.explain_instance(
    instance,
    method="shap",
    generate_text=False,   # NLG îl facem noi în notebook
)

result.keys()


INFO:src.utils:[x] Layer 1: Generating explanations
INFO:src.utils:  method: shap
INFO:src.utils:[x] Layer 2: Normalizing
INFO:src.utils:  features: 30


Raw prediction: 1


dict_keys(['instance', 'method', 'explanation', 'base_value', 'prediction', 'ranked_features', 'statements'])

NLG cu Few-Shot + Ollama

In [5]:
# === Layer 3: NLG cu Ollama (Few-Shot) ===

# generator NLG care folosește llama3:latest din Ollama
nlg = FewShotGenerator(config.nlg, llm_call_fn=ollama_llm_call)

# Top-k features din explicația SHAP
top_k = result["ranked_features"][: config.explainer.top_k_features]
top_k



[('worst area', 0.06724261426271144),
 ('worst concave points', 0.05740842121890832),
 ('mean concave points', 0.04227830540602773),
 ('worst radius', 0.03956915260512325),
 ('worst perimeter', 0.03026387093604719)]

In [6]:

directions = []
for _, v in top_k:
    if v > 0:
        directions.append("supports")
    elif v < 0:
        directions.append("contradicts")
    else:
        directions.append("neutral")

raw_pred = int(result["prediction"])

pred_expert_en = "malignant tumor" if raw_pred == 0 else "benign tumor"
pred_layman_en = "high risk of breast cancer" if raw_pred == 0 else "low risk of breast cancer"

base_context = {
    "features": [f for f, _ in top_k],
    "values": [float(v) for _, v in top_k],
    "directions": directions,
    "method": result.get("method", "shap"),
}


## generăm efectiv textul cu Llama3 prin Ollama

In [7]:
from src.nlg.few_shot_generator import FewShotGenerator
from src.nlg.ollama_client import ollama_llm_call

# puțin mai „cuminte” pentru format fix
config.nlg.temperature = 0.2  

nlg = FewShotGenerator(config.nlg, llm_call_fn=ollama_llm_call)

# 1) context pentru EXPERT
context_expert = {
    **base_context,
    "prediction": pred_expert_en,   # ex: "malignant tumor" / "benign tumor"
    "audience": "expert",
}

# 2) context pentru LAYMAN
context_layman = {
    **base_context,
    "prediction": pred_layman_en,   # ex: "high risk of breast cancer" / "low risk of breast cancer"
    "audience": "layman",
}

text_expert = nlg.generate(context_expert)
text_layman = nlg.generate(context_layman)

print("Explicație tehnică (pentru medic sau cercetător):")
print(text_expert)
print("\nPe scurt, pentru un pacient:")
print(text_layman)


INFO:httpx:HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"


Explicație tehnică (pentru medic sau cercetător):
The model predicts a benign tumor because several morphological features have small positive contributions to the malignancy score. The worst area, concave points, radius, and perimeter all exhibit low values that support the benign class, as reflected by their positive SHAP values. These dominant positive contributions outweigh any potential opposing effects and collectively push the overall prediction towards a benign outcome.

Pe scurt, pentru un pacient:
The model suggests a low risk of breast cancer because the lump's size and shape are more typical of harmless findings. The characteristics that stand out as supportive of this prediction include the lump's relatively small dimensions, which are often seen in non-cancerous cases. Overall, these features point towards a benign outcome.


## Chain of Thought

In [8]:
from src.utils import get_top_k_features

# ranked_features = list of (feature_name, value)
top_k = result["ranked_features"][: config.explainer.top_k_features]

directions = []
for _, v in top_k:
    if v > 0:
        directions.append("supports")
    elif v < 0:
        directions.append("contradicts")
    else:
        directions.append("neutral")

raw_pred = int(result["prediction"])

pred_expert_en = "malignant tumor" if raw_pred == 0 else "benign tumor"
pred_layman_en = "high risk of breast cancer" if raw_pred == 0 else "low risk of breast cancer"

base_context = {
    "features": [f for f, _ in top_k],
    "values": [float(v) for _, v in top_k],
    "directions": directions,
    "method": result.get("method", "shap"),
}


In [9]:
from src.nlg.cot_generator import ChainOfThoughtGenerator
from src.nlg.ollama_client import ollama_llm_call

cot_gen = ChainOfThoughtGenerator(config.nlg, llm_call_fn=ollama_llm_call)

context_cot_expert = {
    **base_context,
    "prediction": pred_expert_en,
}

text_cot_expert = cot_gen.generate(context_cot_expert)

print("=== Chain-of-Thought (technical) ===")
print(text_cot_expert)

context_cot_layman = {
    **base_context,
    "prediction": pred_layman_en,
}

text_cot_layman = cot_gen.generate(context_cot_layman)

print("\n=== Chain-of-Thought (simple) ===")
print(text_cot_layman)



INFO:httpx:HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"


=== Chain-of-Thought (technical) ===
The predicted outcome is that the tumor is benign. The top factors contributing to this prediction are all related to the characteristics of the worst areas and features of the tumor, including its concave points, radius, and perimeter. These factors all support the prediction of a benign tumor, indicating that the tumor's shape and size are consistent with a non-cancerous growth. Overall, the combination of these factors suggests that the tumor is likely to be benign.


INFO:httpx:HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"



=== Chain-of-Thought (simple) ===
The analysis suggests that the patient is at a low risk of developing breast cancer. The key factors contributing to this outcome are the worst area, worst concave points, mean concave points, worst radius, and worst perimeter, all of which support the prediction. These features indicate that the patient's breast tissue has a more compact and uniform structure, with fewer irregularities or abnormalities that could increase the risk of cancer. Overall, the combination of these factors provides strong evidence for a low risk of breast cancer.


## Self_conistency

In [10]:
from src.nlg.self_consistency_generator import SelfConsistencyGenerator

sc_gen = SelfConsistencyGenerator(
    config.nlg,
    n_chains=3,                    # poți pune 3–5, dar ține cont de latență
    llm_call_fn=ollama_llm_call,
)

context_sc_expert = {
    **base_context,
    "prediction": pred_expert_en,
}

text_sc_expert = sc_gen.generate(context_sc_expert)

print("=== Self-Consistency (aggregated, technical) ===")
print(text_sc_expert)

context_sc_layman = {
    **base_context,
    "prediction": pred_layman_en,
}

text_sc_layman = sc_gen.generate(context_sc_layman)

print("\n=== Self-Consistency (aggregated, simple) ===")
print(text_sc_layman)


INFO:httpx:HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"


=== Self-Consistency (aggregated, technical) ===
The model predicts a benign tumor based on several key factors. The worst area and worst concave points of the tumor both strongly support this prediction, indicating that the tumor's shape and size are consistent with a benign growth. Additionally, the mean concave points and worst radius also contribute to this conclusion, suggesting that the tumor's texture and overall structure are typical of a non-cancerous nature. The worst perimeter also supports this finding, implying that the tumor's boundary is not irregular or suspicious. Overall, these factors collectively suggest that the tumor is likely benign.


INFO:httpx:HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"



=== Self-Consistency (aggregated, simple) ===
Based on the model's prediction of a low risk of breast cancer, the top factors that contribute to this outcome are features related to the tumor's shape and size. The presence of a "worst area" with a moderate value of 0.067 supports this prediction, indicating that the tumor is not overly aggressive in terms of its spatial distribution. Similarly, the "worst concave points", "mean concave points", "worst radius", and "worst perimeter" all have positive values, suggesting that the tumor's shape is not particularly irregular or abnormal, which further supports a low risk of breast cancer. Overall, these features collectively contribute to a reassuring profile for the patient.


In [11]:
for tech in ["few_shot", "cot", "self_consistency"]:
    print("\n==============================")
    print(f"TECHNIQUE = {tech}")
    print("==============================")

    res = pipeline.explain_instance(
        instance,
        method="shap",
        generate_text=True,
        technique=tech,        # 👈 AICI testăm ramura
        audience="expert",     # pentru few_shot; la celelalte e ignorat
    )

    print("Reported technique:", res.get("nlg_technique"))
    print("Reported generator:", res.get("nlg_generator"))
    print("\nGenerated text:\n")
    print(res.get("generated_text", "")[:800])  # primele 800 caractere
    print("\nValidation:", res.get("validation"))


INFO:src.utils:[x] Layer 1: Generating explanations
INFO:src.utils:  method: shap
INFO:src.utils:[x] Layer 2: Normalizing
INFO:src.utils:  features: 30
INFO:src.utils:[x] Layer 3: Generating text
INFO:src.utils:  generator: few_shot
INFO:src.utils:  technique: few_shot
INFO:src.utils:  audience: expert



TECHNIQUE = few_shot


INFO:httpx:HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"
INFO:src.utils:[x] Layer 4: Validating
INFO:src.utils:[x] Layer 1: Generating explanations
INFO:src.utils:  method: shap
INFO:src.utils:[x] Layer 2: Normalizing
INFO:src.utils:  features: 30
INFO:src.utils:[x] Layer 3: Generating text
INFO:src.utils:  generator: cot
INFO:src.utils:  technique: cot
INFO:src.utils:  audience: expert


Reported technique: few_shot
Reported generator: few_shot

Generated text:

The model predicts a positive outcome because several morphological features have small to moderate positive contributions to the overall prediction. The worst area, concave points, mean concave points, radius, and perimeter all support this classification with their positive SHAP values. These dominant positive contributions collectively push the overall prediction towards a positive outcome, with no notable opposing factors present in the top attributions.

Validation: {'sum_conservation_valid': False, 'computed_sum': 0.97, 'clarity_score': 78.0}

TECHNIQUE = cot


INFO:httpx:HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"
INFO:src.utils:[x] Layer 4: Validating
INFO:src.utils:[x] Layer 1: Generating explanations
INFO:src.utils:  method: shap
INFO:src.utils:[x] Layer 2: Normalizing
INFO:src.utils:  features: 30
INFO:src.utils:[x] Layer 3: Generating text
INFO:src.utils:  generator: self_consistency
INFO:src.utils:  technique: self_consistency
INFO:src.utils:  audience: expert


Reported technique: cot
Reported generator: cot

Generated text:

The predicted outcome is that the object will be classified as "1". The top factors contributing to this prediction are all related to the shape and size of the object's boundary. Specifically, the worst area, concave points, radius, and perimeter all support the prediction by indicating a complex and irregular shape. These features collectively suggest that the object is not a simple or typical instance, which aligns with the predicted classification of "1".

Validation: {'sum_conservation_valid': False, 'computed_sum': 0.97, 'clarity_score': 83.5}

TECHNIQUE = self_consistency


INFO:httpx:HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"
INFO:src.utils:[x] Layer 4: Validating


Reported technique: self_consistency
Reported generator: self_consistency

Generated text:

The model predicted a score of 1, indicating that this instance is likely to belong to the positive class. The SHAP values reveal that several features contribute positively to this prediction. Specifically, the presence of "worst area", "worst concave points", "mean concave points", "worst radius", and "worst perimeter" all support the predicted outcome. These features collectively suggest that this instance has characteristics that are typical of the positive class.

Validation: {'sum_conservation_valid': False, 'computed_sum': 0.97, 'clarity_score': 85.5}
